Steps to be followed:
1. Obtained the Access Token
2. Obtained the data for spotify
3. Start the data Analysis
4. The recommendation System - based on audio features (content based)
5. The hybrid recommendation System - based on the popularity score based on data preprocessing where popularity weight is calculated.


So to make the API request you  need to following steps:
1. Use your Client Id and Client Secret
2. Create your client creditals which is in the format "client_id:client_secret" 
3. Convert Client creditentials to base64 encode 
4. token Url will be : 'https://accounts.spotify.com/api/token'
5. the format of request will be 
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
6. get the access token

In [ ]:
import base64
import requests
client_Id = ""
client_secret =""
client_credentials = f"{client_Id}:{client_secret}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())
token_url = "https://accounts.spotify.com/api/token"
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response =requests.post(token_url,headers=headers,data=data)

try:
    access_token = response.json().get("access_token")
except:
    print("Access token cannot be obtained")

1. Use spotipy library
2. use SpoitfyoAuth
3. Get the track list which should contain following information - track id, name ,artists, album id and name
4. Create a dictionary for that with these date
5. Get the audio features of the track 
6. Get the release date for the album
7. Get the popularity of the track
8. Add those features to the dictionary and obtained the data
You should have following information in data frame atleast :
Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,


In [27]:
access_token

'BQA7Mgq7UaqJXzM0oINaXhUeKzS5V7D9i8SegXDQQSWRSkEROOqqOULhSw44IbyITGrBsXTNcDrWuHc4ft_c4PLHZOTbAgZA99kgr1d7NKEL8xxZN1Z90BxmJlSu_eE8gb1hivMtYQo'

In [21]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
    print(playlist_tracks)


In [41]:
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

def generate_mock_data(num_tracks=100):
    mock_data = []

    for i in range(num_tracks):
        track_id = str(uuid.uuid4())
        album_id = str(uuid.uuid4())
        track_name = f"Song {i+1}"
        artist = f"Artist {random.randint(1, 5)}"
        album_name = f"Album {random.randint(1, 5)}"
        release_date = (datetime.today() - timedelta(days=random.randint(0, 3000))).strftime("%Y-%m-%d")
        popularity = random.randint(30, 100)
        duration_ms = random.randint(180000, 300000)  # 3 to 5 minutes
        explicit = random.choice([True, False])
        external_url = f"https://open.spotify.com/track/{track_id}"

        audio_features = {
            'Danceability': round(random.uniform(0.3, 0.9), 3),
            'Energy': round(random.uniform(0.3, 0.9), 3),
            'Key': random.randint(0, 11),
            'Loudness': round(random.uniform(-10.0, -3.0), 2),
            'Mode': random.randint(0, 1),
            'Speechiness': round(random.uniform(0.02, 0.1), 3),
            'Acousticness': round(random.uniform(0.1, 0.8), 3),
            'Instrumentalness': round(random.uniform(0.0, 0.6), 3),
            'Liveness': round(random.uniform(0.1, 0.5), 3),
            'Valence': round(random.uniform(0.2, 0.9), 3),
            'Tempo': round(random.uniform(90.0, 160.0), 2)
        }

        track_data = {
            'Track Name': track_name,
            'Artists': artist,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': duration_ms,
            'Explicit': explicit,
            'External URLs': external_url,
            **audio_features
        }

        mock_data.append(track_data)

    df = pd.DataFrame(mock_data)
    return df 
   


In [42]:
df =generate_mock_data()

Data Preprocessing and analysing
1. Check the null values
2. Convert the date time object to number of days
3. For recent popularity create a weight column which is 1/(time_span.days+1)
4. Normalize the music features using MinMax Scaling

In [43]:
df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [44]:
from datetime import datetime
now = datetime.now()
df["weight"] = 1/((now - pd.to_datetime(df["Release Date"], format="%Y-%m-%d")).dt.days)+1

In [49]:
numerical_columns = df.columns[df.dtypes != "object"]

In [56]:
df.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,weight
0,Song 1,Artist 5,Album 5,6ae0c863-cf75-46fd-87d4-5f245b70cc3f,c1fedc9a-2450-44b9-a813-9b24ffce1c28,54,2022-07-16,210632,False,https://open.spotify.com/track/c1fedc9a-2450-4...,...,8,-4.71,1,0.094,0.659,0.205,0.162,0.499,98.74,1.000935
1,Song 2,Artist 2,Album 5,9da9636e-b3b5-4beb-9d1f-07ef6de09ac3,f6c1d8cb-4c81-4cdc-b5e5-a384ae97dfb3,45,2020-10-24,262679,True,https://open.spotify.com/track/f6c1d8cb-4c81-4...,...,1,-6.42,1,0.059,0.758,0.127,0.482,0.600,93.66,1.000589
2,Song 3,Artist 2,Album 1,71ee0d49-7827-4ae6-8906-eef7c5283d22,0720c2ad-9647-46ee-8e7c-e76432f9aead,74,2018-04-01,289137,False,https://open.spotify.com/track/0720c2ad-9647-4...,...,6,-4.39,1,0.093,0.619,0.517,0.434,0.409,117.75,1.000379
3,Song 4,Artist 5,Album 1,37d8c9d6-ad11-4aff-9cab-11aab1f96d70,d77c3a53-9991-47f0-8cb9-8f526cfc8795,58,2021-01-05,280773,True,https://open.spotify.com/track/d77c3a53-9991-4...,...,2,-8.46,0,0.042,0.794,0.528,0.268,0.250,99.47,1.000615
4,Song 5,Artist 4,Album 1,3c7af2b1-cec0-4737-9c57-421ab29f801d,46a31b83-98dd-428b-a29f-af9782ade9e7,34,2019-03-03,266522,False,https://open.spotify.com/track/46a31b83-98dd-4...,...,7,-4.95,0,0.053,0.312,0.148,0.233,0.794,95.62,1.000435


In [54]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_numerical  = scaler.fit_transform(df[numerical_columns])
df_numerical = pd.DataFrame(df_numerical, columns=numerical_columns)
df_numerical["Track Id"] = df["Track ID"]

1. Create a method which accepts the input song name and number of recommendation needs to be given based of the song
2. Get the index of the song in the music df
3.Find the cosine similarity scores based on music features
4. Add that similarity score to the column
5. Get the indices of the similar songs
6. Return the data related to similar songs

In [55]:
df_numerical

,Popularity,Duration (ms),Explicit,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,weight,Track Id
0,0.342857,0.254623,0.0,0.183219,0.247019,0.727273,0.752907,1.0,0.9250,0.799127,0.279926,0.129534,0.429395,0.124820,0.026192,c1fedc9a-2450-44b9-a813-9b24ffce1c28
1,0.214286,0.689726,1.0,0.239726,0.199319,0.090909,0.504360,1.0,0.4875,0.943231,0.136280,0.958549,0.574928,0.051769,0.011058,f6c1d8cb-4c81-4cdc-b5e5-a384ae97dfb3
2,0.628571,0.910910,0.0,0.654110,0.344123,0.545455,0.799419,1.0,0.9125,0.740902,0.854512,0.834197,0.299712,0.398188,0.001930,0720c2ad-9647-46ee-8e7c-e76432f9aead
3,0.400000,0.840988,1.0,0.967466,0.807496,0.181818,0.207849,0.0,0.2750,0.995633,0.874770,0.404145,0.070605,0.135318,0.012211,d77c3a53-9991-47f0-8cb9-8f526cfc8795
4,0.057143,0.721853,0.0,0.282534,0.965928,0.636364,0.718023,0.0,0.4125,0.294032,0.174954,0.313472,0.854467,0.079954,0.004348,46a31b83-98dd-428b-a29f-af9782ade9e7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.014286,0.504280,1.0,0.426370,0.299830,0.636364,0.776163,1.0,0.2875,0.599709,0.141805,0.839378,0.234870,0.649267,0.000961,7893cc3f-72ce-4f80-b4bb-ef272e2a530c
96,0.114286,0.670933,0.0,0.989726,0.843271,0.818182,0.783430,1.0,0.2250,0.289665,0.064457,0.619171,0.328530,0.636468,0.263269,77d73a07-8e95-4636-a94d-b32744b30e0e
97,0.857143,0.161829,0.0,0.256849,0.197615,0.181818,0.536337,1.0,0.3000,0.109170,0.898711,0.989637,1.000000,0.247052,0.004024,237b167f-4b1e-43f1-aa45-1cd814103b52
98,0.357143,0.763050,0.0,0.042808,0.228279,0.818182,0.654070,0.0,0.9375,0.403202,0.517495,0.588083,0.988473,0.000000,0.006558,fccc6121-e5f3-4618-97a9-990fd4c9caaa


In [89]:
from sklearn.metrics.pairwise import cosine_similarity
def song_recommendation(track_id,number_recomendation=5):
    song = df_numerical[df_numerical["Track Id"] == track_id ]
    df_features = df_numerical.drop(columns=["Track Id"])
    song = song.drop(columns =["Track Id"])
    similarity_score = cosine_similarity(song,df_features).flatten()
    df_similarity  = pd.DataFrame({
        "Track Id": df_numerical["Track Id"],
        "Similarity Score": similarity_score
    })
    result_df =df_similarity[df_similarity["Track Id"]!= track_id].sort_values(by=["Similarity Score"],ascending= False)[:number_recomendation]
    result = df[df["Track ID"].isin(result_df["Track Id"])]
    result["Content Score"] = result_df["Similarity Score"]
    return result


In [80]:
song_recommendation_df = song_recommendation("f6c1d8cb-4c81-4cdc-b5e5-a384ae97dfb3")


                                Track Id  Similarity Score
66  ded2db63-080f-48d4-bf12-20bc50e19222          0.916267
87  a00ef035-50d2-4fce-af2f-65aa9c8a89e9          0.900015
95  7893cc3f-72ce-4f80-b4bb-ef272e2a530c          0.891433
22  def888d0-5bfa-4ddb-ab61-70c5a92c714c          0.881821
44  e3603b6c-03ba-4da5-a916-0e12a1195ca8          0.878473


/var/folders/7q/lcztblk90t9f_jg_g52rgx7w0000gn/T/ipykernel_74283/2010399096.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Content Score"] = result_df["Similarity Score"]


1. Create a method called hybrid recommendation which should include three arguments - input song name, number of recommendation , alpha which is the weight you want to give to popularity score
2. Get the content based recommendation song double the number 
3. Find the popularity of the input songs
4. Create the hybrid weighted score - hybrid_recommendations['Hybrid Score'] = (
    alpha * hybrid_recommendations['Content Score'] +
    (1 - alpha) * normalize(hybrid_recommendations['Popularity'])
)
5. Do this for all the recommeded songs as well as the input song
6. Find top five songs with most similar scores

In [92]:
def hybrid_recommendation_system(track_id,number_of_recommendation =5, alpha=0.5):
    content_based_song = song_recommendation(track_id=track_id,number_recomendation=len(df))
    content_based_song["Weighted Popularity Score"] = content_based_song["Popularity"] * content_based_song["weight"]
    content_based_song["Hyrid Weighted Score"] = content_based_song["Content Score"] * alpha +(1-alpha)* content_based_song["Weighted Popularity Score"]
    result  = content_based_song.sort_values(by=["Hyrid Weighted Score"], ascending= False)[:number_of_recommendation]
    return result

In [95]:
hybrid_recommendation_system("f6c1d8cb-4c81-4cdc-b5e5-a384ae97dfb3")[["Track ID","Hyrid Weighted Score","Content Score"]]

/var/folders/7q/lcztblk90t9f_jg_g52rgx7w0000gn/T/ipykernel_74283/2567509778.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Content Score"] = result_df["Similarity Score"]
/var/folders/7q/lcztblk90t9f_jg_g52rgx7w0000gn/T/ipykernel_74283/3162358626.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_based_song["Weighted Popularity Score"] = content_based_song["Popularity"] * content_based_song["weight"]
/var/folders/7q/lcztblk90t9f_jg_g52rgx7w0000gn/T/ipykernel_74283/3162358626.py:4: Set

,Track ID,Hyrid Weighted Score,Content Score
34,c99bfd6d-efc3-4642-b4da-972cacb4e1f5,50.451408,0.783198
27,1016af98-5741-4be5-a136-e69cd7e404e3,50.263682,0.488575
25,2195b103-ac95-46ac-96ad-70b6eefb690f,50.260441,0.481895
40,15cd2c7b-36f5-4148-8cf8-3d9dcf1b2d40,49.942873,0.600443
33,84691178-17fc-4ac8-bfc4-291b16013edd,48.939922,0.683885
